# Selenium Webscraping
## Notebook de clase

En este notebook podras encontrar el ejercicio que hicimos en clase para aprender las bases de Selenium.

Adicionalmente, se puede consultar la documentación oficial: https://www.selenium.dev/documentation/webdriver/

Hagamos los imports relevantes y conectémonos al driver de Chrome. 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Remote(command_executor='http://localhost:4444/wd/hub',options = webdriver.ChromeOptions())

In [19]:

#Por si falla algo. URL al driver de Chrome (descargar el driver desde https://sites.google.com/chromium.org/driver/)
driver = webdriver.Chrome(options = webdriver.ChromeOptions())

Utilizaremos un sitio web de aprendizaje. Nota: en la vida diaria los sitios se van actualizando, lo cual puede dificultar mantener el código al día y funcionando 

In [20]:
url = 'https://scrapepark.org/courses/spanish/'

# Abrir la página web
driver.get(url)

No es necesario que se abra la ventana de Google, pero al principio es muy útil para saber qué está pasando exactamente con nuestra automatización. Una vez que terminamos de debuggear, es recomendable utilizar el driver sin visualizar la ventana para ahorrar RAM y hacer más rápido los procesos.

In [ ]:
options.add_argument('--headless')  # Ejecutar en modo headless (sin interfaz gráfica)
driver = webdriver.Remote(command_executor='http://localhost:4444/wd/hub',options= webdriver.ChromeOptions())


En este caso es importante cerrar las sesiones con el siguiente comando:

In [7]:
driver.close()

## **Scrapeo básico** 

 ``driver.find_element(By.<>,'html')``


In [ ]:
nom_pagina = driver.title #titulo de la página
print("El título de la página es:", nom_pagina)


In [ ]:

print('\n')

info_marca = driver.find_element(By.CLASS_NAME, 'information-f')
print("La información del sitio es:", info_marca.text) # .text nos da el texto de un elemento

print('\n')

testimonio = driver.find_element(By.ID, 'testimonios') # buequda por ID
print("El texto es:", testimonio.text) # .text nos da el texto de un elemento


¿qué pasa si queremos el testimonio de todos los clientes? Necesitamos interectuar con el sitio a través de pithon

In [ ]:

testimonio = driver.find_element(By.ID, 'testimonios')
print("El texto es:", testimonio.text) # .text nos da el texto de un elemento
print('\n')

next = driver.find_element(By.CLASS_NAME, 'carousel-control-next')
next.click()

testimonio = driver.find_element(By.ID, 'testimonios')
print("El texto es:", testimonio.text) # .text nos da el texto de un elemento



## paciencia
en este caso se tiene que tomar en cuenta la velocidad del click contra la velocidad de extraccion y definición de un elemento

In [22]:
import time

testimonio = driver.find_element(By.ID, 'testimonios')
print(testimonio.text)
print('\n')

next = driver.find_element(By.CLASS_NAME, 'carousel-control-next')
next.click()

time.sleep(.5) #espera medio segundo

testimonio = driver.find_element(By.ID, 'testimonios')
print(testimonio.text) 
next.click()

time.sleep(.5) #espera medio segundo

testimonio = driver.find_element(By.ID, 'testimonios')
print(testimonio.text) 


Testimonios de clientes
Cliente 3
Estoy muy conforme. Hay muchas patinetas y los diseños son fantásticos.
Anterior
Siguiente


Testimonios de clientes
Cliente 1
Los productos me encantaron y los precios son muy buenos. Lo recomiendo.
Anterior
Siguiente
Testimonios de clientes
Cliente 2
¡La calidad y variedad de patinetas es impresionante! Definitivamente volveré a comprar.
Anterior
Siguiente


¿Cómo localizar un objeto usando su Xpath.

Ventaja: efectividad; 
Desventaja: trabajo

Xpath Syntax:
//tagName[@AttributeName="Value"]

In [23]:
#//a[@href= "#productos"]
botonProductos= driver.find_element(By.XPATH,'//a[@href= "#productos"]')
#botonProductos= driver.find(By.XPATH('//*[@id="navbarSupportedContent"]/ul/li[3]/a'))# Ej. No funciona
botonProductos.click()


In [24]:
detailBoxes= driver.find_elements(By.XPATH,'//div[@class= "detail-box"]') #detail boxes es una lista

for box in detailBoxes:
    print(box.text)



Descuentos 20% Off
¡En todos los productos!
Aprovechá nuestras ofertas.
Comprar ahora

Entrega rápida
Envío gratuito
La mejor calidad
Patineta Nueva 1
$75
Patineta Usada 2
$80
Patineta Nueva 3
$68
Patineta Usada 4
$70
Patineta Nueva 5
$75
Patineta Nueva 6
$58
Patineta Nueva 7
$80
Patineta Nueva 8
$35
Patineta Nueva 9
$165
Patineta Usada 10
$54
Patineta Usada 11
$99
Patineta Nueva 12
$110

Cliente 2
¡La calidad y variedad de patinetas es impresionante! Definitivamente volveré a comprar.



In [25]:
#Queremos crear un data frame de los productos con sus precios 
#usaremos regex para obtener solo el texto de las boxes que necesitamos
import re

patronProducto= r"Patineta (Nueva|Usada) \d{1,2}"

#arreglo de productos
matches = [box for box in detailBoxes if re.search(patronProducto, box.text)]

for box in matches:
    print(box.text)


Patineta Nueva 1
$75
Patineta Usada 2
$80
Patineta Nueva 3
$68
Patineta Usada 4
$70
Patineta Nueva 5
$75
Patineta Nueva 6
$58
Patineta Nueva 7
$80
Patineta Nueva 8
$35
Patineta Nueva 9
$165
Patineta Usada 10
$54
Patineta Usada 11
$99
Patineta Nueva 12
$110


In [ ]:
import pandas as pd

pattern = r"Patineta (Nueva|Usada) (\d{1,2})\s*\n*\s*\$(\d+)" #ahora más específico

# Crear listas para almacenar los productos, estados, números y precios
productos = [] #un producto es el estado de la patineta con su número
estados = []
numeros = []
precios = []

# Iteramos
for match in matches:
    resultado = re.search(pattern, match.text)
    if resultado:
        #separamos y definimos variables
        tipo = resultado.group(1) + (resultado.group(2))
        estado = resultado.group(1) 
        numero = int(resultado.group(2))
        precio = int(resultado.group(3))
        
        #agregamos a las listas
        productos.append(tipo)
        estados.append(estado)
        numeros.append(numero)
        precios.append(precio)

# Crear el DataFrame usando pandas
data = {
    'Producto': productos,
    'Estado': estados,
    'Número': numeros,
    'Precio': precios
}

df = pd.DataFrame(data)

print(df)

Ya treniendo el data frame podemos usar las funciones de pandas para analizar los datos

In [ ]:
print('Precio promedio:',df['Precio'].mean())
print('Precio mínimo:',df['Precio'].min())
print('Precio máximo',df['Precio'].max())

In [ ]:
print('Precio promedio de las patinetas usadas', df[df['Estado'] == 'Usada']['Precio'].mean())
print('Precio promedio de las patinetas nuevas', df[df['Estado'] == 'Nueva']['Precio'].mean())


print('Rango de precios de las patinetas usadas', df[df['Estado'] == 'Usada']['Precio'].max()-df[df['Estado'] == 'Usada']['Precio'].min())
print('Rango de precios de las patinetas nuevas', df[df['Estado'] == 'Nueva']['Precio'].max()-df[df['Estado'] == 'Nueva']['Precio'].min())

In [ ]:
# Cerrar el navegador
#driver.quit()

